References:<br>
https://www.youtube.com/watch?v=_MZf85lseec<br>
https://forms.cloud.microsoft/r/uDEzCqEGSw<br>

In [1]:
import pandas as pd
from skollama.models.ollama.classification.zero_shot import ZeroShotOllamaClassifier

# Parameters

In [2]:
COMMOM_COLS = [
    'ID',
    'Start time',
    'Completion time',
    'Email',
    'Name',
    'Last modified time',
]
SINGLE_ANSWER_QUESTIONS_COLS = [
    'How often do you order pizza from our pizzeria?',
    'What did you like the most or the least about your last pizza order?',
    'Please describe your most recent experience ordering from our pizzeria.',
    'How would you rate the quality of our pizza?',
    'When was the last time you ordered from our pizzeria?',
    'How likely are you to recommend our pizzeria to a friend or colleague?',
]
MULTIPLE_ANSWER_QUESTION_1_COLS = [
    'Which of the following pizza toppings do you usually choose?',
]
MATRIX_ANSWER_QUESTION_1_COLS = [
    'Pizza taste',
    'Delivery speed',
    'Customer service',
    'Value for money',
]
RANKING_ANSWER_QUESTION_1_COLS = [
    'Please rank the following aspects of our service from most important to least important.',
]
SHORT_LABELS = [
    'positive',
    'neutral',
    'negative',
]
LONG_LABELS = [
    'food',
    'service',
    'kids area',
    'price',
    'ambience',
    'cleanliness',
    'location',
    'variety',
    'speed',
    'quality',
    'kids area and entertainment',
    'overall experience',
]

# Classification

In [3]:
classifier_short = ZeroShotOllamaClassifier(model='phi3')
classifier_long = ZeroShotOllamaClassifier(model='phi3')
classifier_short.fit(None, SHORT_LABELS)
classifier_long.fit(None, LONG_LABELS)

,model,'phi3'
,host,'http://localhost:11434'
,options,None
,default_label,'Random'
,prompt_template,None
,structured_output,''


In [4]:
answers = pd.read_excel('../assets/pizza_raw.xlsx')

In [5]:
answers

,ID,Start time,Completion time,Email,Name,Last modified time,How often do you order pizza from our pizzeria?,Which of the following pizza toppings do you usually choose?,What did you like the most or the least about your last pizza order?,Please describe your most recent experience ordering from our pizzeria.,How would you rate the quality of our pizza?,When was the last time you ordered from our pizzeria?,Please rank the following aspects of our service from most important to least important.,Pizza taste,Delivery speed,Customer service,Value for money,How likely are you to recommend our pizzeria to a friend or colleague?
0,1,2025-11-04 17:32:00,2025-11-04 17:33:00,anonymous,NaN,NaN,Rarely,Four Cheese;Margherita;Veggie;,Perfect experience from order to delivery.,The app was easy to use and order tracking wor...,4,2025-10-28,Delivery time;Taste;Price;,Dissatisfied,Dissatisfied,Satisfied,Very Satisfied,8
1,2,2025-11-04 17:34:00,2025-11-04 17:36:00,anonymous,NaN,NaN,Rarely,Margherita;,"Average taste, nothing special.",The crust was hard and not fresh.,3,2025-10-25,Taste;Price;Delivery time;,Very Satisfied,Very Satisfied,Very Dissatisfied,Satisfied,9
2,3,2025-11-04 17:36:00,2025-11-04 17:37:00,anonymous,NaN,NaN,Rarely,Veggie;Four Cheese;Chicken BBQ;,Perfectly balanced taste and temperature.,Delivery was quick and pizza was still hot.,3,2025-10-28,Price;Taste;Delivery time;,Very Dissatisfied,Satisfied,Very Dissatisfied,Very Satisfied,5
3,4,2025-11-04 17:38:00,2025-11-04 17:40:00,anonymous,NaN,NaN,Twice a month,Margherita;Veggie;Chicken BBQ;,Excellent customer service!,Amazing service from staff.,1,2025-10-25,Price;Taste;Delivery time;,Very Satisfied,Very Dissatisfied,Very Dissatisfied,Very Dissatisfied,8
4,5,2025-11-04 17:40:00,2025-11-04 17:43:00,anonymous,NaN,NaN,Once a week,Pepperoni;Four Cheese;Veggie;,"Average taste, nothing special.",Always a pleasant experience ordering here.,5,2025-10-29,Taste;Price;Delivery time;,Dissatisfied,Neutral,Satisfied,Dissatisfied,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,2025-11-04 20:42:00,2025-11-04 20:44:00,anonymous,NaN,NaN,Twice a month,Veggie;,"The cheese was amazing, will order again.",Always a pleasant experience ordering here.,3,2025-10-31,Delivery time;Taste;Price;,Dissatisfied,Very Satisfied,Dissatisfied,Neutral,10
96,97,2025-11-04 20:44:00,2025-11-04 20:47:00,anonymous,NaN,NaN,Rarely,Pepperoni;Margherita;,Crust was too thick for my liking.,Delivery was quick and pizza was still hot.,2,2025-10-30,Delivery time;Taste;Price;,Satisfied,Dissatisfied,Satisfied,Very Dissatisfied,3
97,98,2025-11-04 20:46:00,2025-11-04 20:49:00,anonymous,NaN,NaN,Twice a month,Margherita;,Perfect experience from order to delivery.,The app was easy to use and order tracking wor...,4,2025-10-25,Taste;Delivery time;Price;,Dissatisfied,Very Satisfied,Dissatisfied,Very Dissatisfied,8
98,99,2025-11-04 20:48:00,2025-11-04 20:49:00,anonymous,NaN,NaN,Twice a month,Veggie;Four Cheese;,"The cheese was amazing, will order again.",The crust was hard and not fresh.,4,2025-10-29,Taste;Price;Delivery time;,Neutral,Very Satisfied,Very Dissatisfied,Satisfied,4


In [6]:
answers.insert(
    loc = 0,
    column = 'What did you like the most or the least about your last pizza order? (Sentiment)',
    value = classifier_short.predict(answers['What did you like the most or the least about your last pizza order?'])
)

100%|██████████| 100/100 [02:15<00:00,  1.36s/it]


In [7]:
answers.insert(
    loc = 0,
    column = 'Please describe your most recent experience ordering from our pizzeria. (Category)',
    value = classifier_long.predict(answers['Please describe your most recent experience ordering from our pizzeria.'])
)

100%|██████████| 100/100 [02:06<00:00,  1.26s/it]


# Structure

In [8]:
fact_answers = answers[COMMOM_COLS + SINGLE_ANSWER_QUESTIONS_COLS].copy()

In [9]:
dim_choice_question_1 = answers[['ID'] + MULTIPLE_ANSWER_QUESTION_1_COLS].copy()

In [10]:
dim_ranking_question_1 = answers[['ID'] + RANKING_ANSWER_QUESTION_1_COLS].copy()

In [11]:
dim_matrix_question_1 = answers[['ID'] + MATRIX_ANSWER_QUESTION_1_COLS].copy()

## Choice question 1

In [12]:
def split(array: str, sep = ';'):
    try:
        _array = array.split(sep)
        _array.remove('')
    except Exception:
        _array = []
    return _array

In [16]:
dim_choice_question_1.loc[:, ('Which of the following pizza toppings do you usually choose?')] = dim_choice_question_1.loc[:, ('Which of the following pizza toppings do you usually choose?')].apply(split)

In [17]:
dim_choice_question_1 = dim_choice_question_1.explode('Which of the following pizza toppings do you usually choose?', ignore_index=True)

## Ranking question 1

In [20]:
def rank(array: str, sep = ';'):
    try:
        _array = array.split(sep)
        _array.remove('')
        ranked = [(item, rank + 1) for rank, item in enumerate(_array)]
    except Exception:
        ranked = []
    return ranked

In [21]:
def select(tuple, index):
    try:
        return tuple[index]
    except Exception:
        return None

In [22]:
dim_ranking_question_1.loc[:, ('Please rank the following aspects of our service from most important to least important.')] = dim_ranking_question_1.loc[:, ('Please rank the following aspects of our service from most important to least important.')].apply(rank)

In [23]:
dim_ranking_question_1 = dim_ranking_question_1.explode('Please rank the following aspects of our service from most important to least important.', ignore_index=True)

In [24]:
dim_ranking_question_1.insert(
    loc = len(dim_ranking_question_1.columns),
    column = 'Please rank the following aspects of our service from most important to least important. (Option)',
    value = dim_ranking_question_1.loc[:, ('Please rank the following aspects of our service from most important to least important.')].apply(lambda x: select(x, 0))
)

In [25]:
dim_ranking_question_1.insert(
    loc = len(dim_ranking_question_1.columns),
    column = 'Please rank the following aspects of our service from most important to least important. (Value)',
    value = dim_ranking_question_1.loc[:, ('Please rank the following aspects of our service from most important to least important. (Option)')].apply(lambda x: select(x, 1))
)

In [26]:
dim_ranking_question_1.drop(columns=['Please rank the following aspects of our service from most important to least important.'], inplace=True)

## Matrix question 1

In [27]:
dim_matrix_question_1 = dim_matrix_question_1.melt(
    id_vars=['ID'],
    value_vars=MATRIX_ANSWER_QUESTION_1_COLS,
    var_name='Statement',
    value_name='Answer'
).dropna(subset=['Answer']).reset_index(drop=True)

# Export

In [29]:
with pd.ExcelWriter('../assets/pizza_processed.xlsx') as writer:
    fact_answers.to_excel(writer, sheet_name='Fact Answers', index=False)
    dim_choice_question_1.to_excel(writer, sheet_name='Dim Choice Question 1', index=False)
    dim_ranking_question_1.to_excel(writer, sheet_name='Dim Ranking Question 1', index=False)
    dim_matrix_question_1.to_excel(writer, sheet_name='Dim Matrix Question 1', index=False)